# Capstone Project

In [124]:
import requests
import random
import pandas as pd
import numpy as np
import sqlite3 as lite
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import time
import datetime
from dateutil.parser import parse
import collections
import json
import ijson
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics
from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold

## Try some tactics to deal with huge file

In [50]:
# %%bash
# head -1 yelp_academic_dataset_review.json

In [51]:
# %%bash
# head -1 yelp_academic_dataset_review.json > output.txt

In [52]:
# %%bash
# echo -e output.csv

In [53]:
# %%bash
# cat yelp_academic_dataset_review.json | cut -d \, -f 6 > stars.csv

In [54]:
# %%bash
# cat yelp_academic_dataset_review.json | cut -d \, -f 8 > review.txt

In [55]:
# %%bash
# cat yelp_academic_dataset_review.json | cut -d \: -f 10 > review.csv

In [56]:
# %%bash

# grep -e '^{"' yelp_academic_dataset_review.json

In [57]:
# with open('yelp_academic_dataset_review.json','rU') as f:
#     data = json.load(f)

In [58]:
# for prefix, the_type, value in ijson.parse(open('yelp_academic_dataset_review.json')):
# #     if the_type == 'map_key':
# #         if value == 'stars':
# #         print the_type, value
#     print prefix, the_type, value

In [59]:
# df = pd.read_csv('stars.csv',names=['Stars','Text'])

In [60]:
# df['Stars'] = df['Stars'].map(lambda x: str(x)[-1])

In [61]:
# df

In [62]:
# df2 = pd.read_csv('review.csv')

In [63]:
# parser = ijson.parse(open('yelp_academic_dataset_review.json'))
# reviews = []
# for prefix, event, value in parser:
#     if (prefix, event) == ('start_map','map_key'):
#         reviews.append(value)
# # stream.write('</yelp>')
# reviews

In [64]:
# def load_json(filename):
#     with open(filename, 'r') as fd:
#         parser = ijson.parse(fd)
#         ret = {'stars':{}}
#         for prefix, event, value in parser:
#             if (prefix, event) == ('stars', 'map_key'):
#                 star_amount = value
#                 ret['stars'][star_amount] = {}
#         return ret

In [65]:
# df = pd.DataFrame(columns=['stars','text'])

# with open('yelp_academic_dataset_review.json','rU') as f:
#     for line in f:
#         data = json.loads(line)
        
# result = json_normalize(data, ['stars', 'text'])
# # df['stars'] = data['stars']
# # df['text'] = data['text']

## Iteratively load json's line by line

In [66]:
# data = []
# for line in open('yelp_academic_dataset_review.json','rU'):
#     data.append(json.loads(line))
# data

In [67]:
data = []
with open('yelp_academic_dataset_review.json','rU') as reviews:
#     i = 0
#     while i < 10000:
    
    first_line = json.loads(reviews.readline())
    print (first_line['stars'], first_line['text'])
#         i += 1


(4, u'Mr Hoagie is an institution. Walking in, it does seem like a throwback to 30 years ago, old fashioned menu board, booths out of the 70s, and a large selection of food. Their speciality is the Italian Hoagie, and it is voted the best in the area year after year. I usually order the burger, while the patties are obviously cooked from frozen, all of the other ingredients are very fresh. Overall, its a good alternative to Subway, which is down the road.')


## Create training set from first 25,000 reviews, testing set from the next 5,000 reviews:

- Create even split

In [68]:
# data = []
# with open('yelp_academic_dataset_review.json','rU') as reviews:
#     i = 0
#     while i < 10000:
#         first_line = json.loads(reviews.readline())
#         if first_line['stars'] != 3:
#             data.append([first_line['stars'], first_line['text']])
#         i += 1
# # data

In [69]:
data = []
data_test = []
y_train = []
y_test = []
with open('yelp_academic_dataset_review.json','rU') as reviews:
    i = 0
    while i < 25000:
        first_line = json.loads(reviews.readline())
        if first_line['stars'] != 3:
            if first_line['stars'] < 3:
                first_line['stars'] = 0
            else:
                first_line['stars'] = 1
            data.append([first_line['stars'], first_line['text']])
            y_train.append(first_line['stars'])
        i += 1
    while 24999 <= i < 30000:
        first_line = json.loads(reviews.readline())
        if first_line['stars'] != 3:
            if first_line['stars'] < 3:
                first_line['stars'] = 0
            else:
                first_line['stars'] = 1
            data_test.append([first_line['stars'], first_line['text']])
            y_test.append(first_line['stars'])
        i += 1

# data_test

In [70]:
len_y = len(y_train)
temp = [i for i in y_train if i == 0]
temp_0 = temp.count(0)
percentage = 100*(float(temp_0)/float(len_y))
percentage, temp_0

(25.077517530887754, 5257)

In [71]:
temp = [i for i in y_train if i == 1]
temp_1 = temp.count(1)
percentage = 100*(float(temp_1)/float(len_y))
percentage, temp_1

(74.92248246911225, 15706)

In [154]:
k_fold_prep = []
k_fold_prep_stars = []
X = []
# X = np.array(X)
with open('yelp_academic_dataset_review.json','rU') as reviews:
    i = 0
    while i < 100:
        first_line = json.loads(reviews.readline())
        if first_line['stars'] != 3:
            if first_line['stars'] < 3:
                first_line['stars'] = 0
            else:
                first_line['stars'] = 1
            k_fold_prep.append([first_line['stars'], first_line['text']])
            k_fold_prep_stars.append(first_line['stars'])
            X.append(first_line['text'])
        i += 1


In [143]:
# k_fold_prep_stars = []
# for i, j in k_fold_prep:
#     k_fold_prep_stars.append(k_fold_prep[i][0])
k_fold_prep_stars

[1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1]

In [137]:
k_fold_prep[1][0]

1

In [74]:
stars_0 = []
stars_1 = []
X = []
y = []
for review in k_fold_prep:
    if review[0] == 0:
        stars_0.append(review)
    elif review[0] == 1:
        stars_1.append(review)
    X.append(review[1])
    y.append(review[0])

In [75]:
len_y = len(y)
temp = [i for i in y if i == 0]
temp_1 = temp.count(0)
percentage = 100*(float(temp_1)/float(len_y))
percentage, temp_1

(25.0, 21)

In [76]:
num_to_select = 10
randos = random.sample(stars_0, num_to_select)
randos

[[0,
  u'Wing sauce is like water. Pretty much a lot of butter and some hot sauce (franks red hot maybe).  The whole wings are good size and crispy, but for $1 a wing the sauce could be better. The hot and extra hot are about the same flavor/heat.  The fish sandwich is good and is a large portion, sides are decent.'],
 [0,
  u"Waited over 45 mins, 2 people ahead of me and I'm finally getting helped...  Wtf???\nI can understand them being busy but that was ridiculous.  I work out in Robinson and went to the Settlers Ridge location and BOY- WHAT A DIFFERENCE.  They were awesome.  In & out the door in 20 minutes- I ended up with a new 6s and my wife ordered a 6sPlus... All in less than 20 minutes!  I'll never go to this store anymore."],
 [0,
  u'Station is closed.  Sign is blank and all fixtures from the store have been removed.'],
 [0,
  u'I can;t believe this place has a high rating. I had a bichon, who was seen here since his puppyhood and who was dying. I called to make an appointmen

In [78]:
# rando = np.random.choice(stars_0, replace=False)

In [79]:
kf = KFold(len(k_fold_prep), n_folds=10)
for train, test in kf:
    print 'TRAIN:', train, 'TEST:', test
#     X_train, X_test = X[train], X[test]
#     y_train, y_test = y[train], y[test]

TRAIN: [ 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33
 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58
 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83] TEST: [0 1 2 3 4 5 6 7 8]
TRAIN: [ 0  1  2  3  4  5  6  7  8 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33
 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58
 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83] TEST: [ 9 10 11 12 13 14 15 16 17]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 27 28 29 30 31 32 33
 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58
 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83] TEST: [18 19 20 21 22 23 24 25 26]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58
 59 60 61 62 63 64 65 66 67 68 69 70 

In [145]:
skf = StratifiedKFold(k_fold_prep_stars, n_folds=10)
len(skf), skf

(10,
 sklearn.cross_validation.StratifiedKFold(labels=[1 1 1 1 0 1 1 1 0 0 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
  1 1 1 1 1 0 1 1 1 1 1 0 0 0 1 0 1 1 1 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 0 1 1
  0 1 0 1 1 0 0 0 1 1], n_folds=10, shuffle=False, random_state=None))

In [158]:
X[10]

u"Before I finally made it over to this range I heard the same thing from most people - it's just fine to go work on your swing. I had such a low expectation I was pleasantly surprised. \n\nIt's a fairly big range - if you are familiar with Scally's in Moon, it seems like it has almost as many tees, though its not nearly as nice a facility. \n\nThe guys in the pro shop were two of the friendlier guys I've come across at ranges or at courses. Yards were indeed marked and there are some targets to aim for, and even some hazards to aim away from. \n\nA big red flag to me was the extra charge ($3) to hit off the grass. I am no range expert, but this is the 4th one I've been to and the first I've seen of that sort of nickel and diming....\n\nPrice for the golf balls was reasonable and I do plan to be back every week until they close up in October for the season. Hopefully, since its for sale, it will reopen as a golf facility again."

In [156]:
for train, test in skf:
    print 'TRAIN:', train, 'TEST:', test
    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]

TRAIN: [10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34
 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83] TEST: [0 1 2 3 4 5 6 7 8 9]


TypeError: only integer arrays with one element can be converted to an index

## Sort reviews by star number, make dataframes:

In [80]:
df = pd.DataFrame(data, columns = ['stars','text']).sort('stars')
df.reset_index(drop=True, inplace=True)

In [81]:
df.head()

,stars,text
0,0,Pretty poor experience here. We were seated qu...
1,0,"Spoke with a self appointed ""big cheese"" at th..."
2,0,"One word, ugh. In dire need of upgrades. Dirty..."
3,0,The worst dry cleaners ever with the shittiest...
4,0,I'm writing this review as I sit in the Hilton...


In [82]:
df_testing = pd.DataFrame(data_test, columns = ['stars','text']).sort('stars')
df_testing.reset_index(drop=True, inplace=True)

In [83]:
df_testing.head()

,stars,text
0,0,I realize this is a pizza chain but that doesn...
1,0,I thought I liked this place but I have change...
2,0,CAN I RATE LESS THAN A STAR?\nMy first and onl...
3,0,Literally the worst Chinese buffet I have ever...
4,0,Went a second time and did not have a great ex...


## Clean text for Bag-of-Words:

In [84]:
example1 = BeautifulSoup(df['text'][75])
example1.get_text()

/Users/jacobmink/anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


u'Im soo surprised with the food that we ate. Im from California and was excited to experience this restaurant. Especially, since it is a local fav. I ordered the house made spaghetti expecting a fresh tomato base sauce. Unfortunately what I got tasted like a canned sauce with minced onions. It had no...flavor!  Now maybe I ruined the meal by ordering gluten free noodles....but I didnt think that would make it this bad. Im soo sorry because our server was awesome and they even accommodated our big party. On a good note, the bread was delicious.\nAs much as I hate Olive Garden...I would preferred that.'

In [85]:
def review_to_words(raw_review):
    #     1) Remove HTML:
    review_text = BeautifulSoup(raw_review).get_text()
    #     2) Remove non-letters:
    letters_only = re.sub('[^a-zA-Z]',' ', review_text)
    #     3) Convert to lower case, split into words:
    words = letters_only.lower().split()
    #     4) Convert stopwords to set:
    stops = set(stopwords.words('english'))
    #     5) Remove stopwords:
    meaningful_words = [w for w in words if not w in stops]
    #     6) Join words back into one string separated by space:
    return( ' '.join(meaningful_words))

In [86]:
clean_example = review_to_words(df['text'][0])
clean_example

u'pretty poor experience seated quickly waitstaff seemed unfriendly ordered truffle fries traditional aoili truffle sauce covered fries steakfries chipotle ranch dipping sauce impressed average hamburger chicken tenders nothing special fairly expensive forgettable'

In [87]:
num_reviews = df['text'].size
num_reviews

20963

In [88]:
num_test_reviews = df_testing['text'].size
num_test_reviews

4191

In [89]:
print "Cleaning and parsing the training set Yelp reviews...\n"
clean_train_reviews = []
for i in xrange(0, num_reviews):
    if (i+1)%1000 == 0:
        print "Review %d of %d\n" % (i+1,num_reviews)
    clean_train_reviews.append(review_to_words(df['text'][i]))

Cleaning and parsing the training set Yelp reviews...

Review 1000 of 20963

Review 2000 of 20963

Review 3000 of 20963

Review 4000 of 20963

Review 5000 of 20963

Review 6000 of 20963

Review 7000 of 20963

Review 8000 of 20963

Review 9000 of 20963

Review 10000 of 20963

Review 11000 of 20963

Review 12000 of 20963

Review 13000 of 20963

Review 14000 of 20963

Review 15000 of 20963

Review 16000 of 20963

Review 17000 of 20963

Review 18000 of 20963

Review 19000 of 20963

Review 20000 of 20963



In [90]:
print "Cleaning and parsing the testing set Yelp reviews...\n"
clean_test_reviews = []
for i in xrange(0, num_test_reviews):
    if (i+1)%1000 == 0:
        print "Review %d of %d\n" % (i+1,num_test_reviews)
    clean_test_reviews.append(review_to_words(df_testing['text'][i]))

Cleaning and parsing the testing set Yelp reviews...

Review 1000 of 4191

Review 2000 of 4191

Review 3000 of 4191

Review 4000 of 4191



In [91]:
clean_test_reviews[0]

u'realize pizza chain make substandard service ordered week ago delivery person phone sure specials asked getting specials placed order pepperoni hot wings pizza arrived hour told minutes driver less cordial handed pizza mumbled price bringing pizza inside opening box pizza cold wings since hungry want call back deal phone person sucked ate pieces pizza wings pizza flavorless crust tasted like cardboard wings slimy chewy needless say never go back papa john going stick local pizzeria danny pizza street'

In [92]:
clean_train_reviews[0]

u'pretty poor experience seated quickly waitstaff seemed unfriendly ordered truffle fries traditional aoili truffle sauce covered fries steakfries chipotle ranch dipping sauce impressed average hamburger chicken tenders nothing special fairly expensive forgettable'

## Vectorize word counts, train on training set:

In [93]:
vectorizer = CountVectorizer(analyzer='word',
                             tokenizer=None,
                             preprocessor=None,
                             stop_words=None,
                             max_features=5000)

train_data_features = vectorizer.fit_transform(clean_train_reviews)
train_data_features = train_data_features.toarray()

## Vectorize testing set words:

In [94]:
test_data_features = vectorizer.transform(clean_test_reviews)
test_data_features = test_data_features.toarray()

In [95]:
test_data_features.shape

(4191, 5000)

In [96]:
train_data_features.shape

(20963, 5000)

In [97]:
vocab = vectorizer.get_feature_names()
vocab[1:20]

[u'ability',
 u'able',
 u'absolute',
 u'absolutely',
 u'ac',
 u'accent',
 u'accept',
 u'acceptable',
 u'accepted',
 u'access',
 u'accessible',
 u'accessories',
 u'accident',
 u'accidentally',
 u'accommodate',
 u'accommodated',
 u'accommodating',
 u'accomodating',
 u'accompanied']

In [98]:
dist = np.sum(train_data_features, axis=0)
for tag, count in zip(vocab[1:20], dist):
    print count, tag

25 ability
40 able
920 absolute
141 absolutely
883 ac
52 accent
29 accept
98 acceptable
70 accepted
38 access
137 accessible
44 accessories
31 accident
50 accidentally
30 accommodate
72 accommodated
27 accommodating
204 accomodating
26 accompanied


## Generate random forest model:

- Include weighting to account for imbalanced classes in training set

In [112]:
print 'Training the random forest...'

forest = RandomForestClassifier(n_estimators=100)
forest = forest.fit(train_data_features,
                    df['stars'],
                    sample_weight=np.array([3 if i == 0 else 1 for i in df['stars']]))

Training the random forest...


In [113]:
print 'Testing using the random forest...'
result = forest.predict(test_data_features)
output = pd.DataFrame(data={'text':df_testing['text'],'stars_pred':result})

Testing using the random forest...


In [114]:
output

,stars_pred,text
0,0,I realize this is a pizza chain but that doesn...
1,0,I thought I liked this place but I have change...
2,0,CAN I RATE LESS THAN A STAR?\nMy first and onl...
3,0,Literally the worst Chinese buffet I have ever...
4,1,Went a second time and did not have a great ex...
5,1,"Not the worst Chinese I've had, but definitely..."
6,0,These Ladys where allllllll up in my business....
7,0,"again, came here as it was within walking dist..."
8,0,I wish i could say good things about this busi...
9,0,Greasy unappetizing fried food. Low quality in...


## Evaluate model predictions:

- Sensitivity and specificity analysis, check bias

- Create a zero rate classifier

- f1 Score

In [115]:
precision = metrics.precision_score(df_testing['stars'],result)
recall = metrics.recall_score(df_testing['stars'],result)
precision, recall

(0.89134438305709029, 0.96832277425808599)

In [116]:
f1_score = metrics.f1_score(df_testing['stars'],result)
f1_score

0.92824037078472099

In [117]:
# class_frequency = 

In [118]:
# accuracy = 

In [119]:
test_process = result == df_testing['stars']
test_process.value_counts()

True     3742
False     449
dtype: int64

In [120]:
df['stars'].value_counts()

1    15706
0     5257
dtype: int64

In [121]:
difference = result - df_testing['stars']
difference

0       0
1       0
2       0
3       0
4       1
5       1
6       0
7       0
8       0
9       0
10      1
11      0
12      1
13      0
14      0
15      1
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      0
27      0
28      0
29      1
       ..
4161    0
4162    0
4163    0
4164    0
4165    0
4166    0
4167    0
4168    0
4169   -1
4170    0
4171    0
4172    0
4173    0
4174    0
4175    0
4176    0
4177    0
4178    0
4179    0
4180    0
4181    0
4182    0
4183    0
4184    0
4185    0
4186    0
4187    0
4188    0
4189    0
4190    0
Name: stars, dtype: int64

In [122]:
length = float(len(df_testing['stars']))

In [123]:
mse = sum((difference)**2)/length
mse

0.10713433548079217